In [ ]:
import torch
import torch.nn as nn
import numpy as np


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, alpha):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.alpha = alpha
        self.optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True
        )[0]
        u_x, u_t = u_x_t[:, 0], u_x_t[:, 1]
        u_xx = torch.autograd.grad(u_x, g, torch.ones_like(u_x), create_graph=True)[0][
            :, 0
        ]
        residual = u_t - self.alpha * u_xx
        return self.loss(residual, fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], device="cuda")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}")
        return loss.item()

In [6]:
import numpy as np
import h5py
L = 1.0
Nx = 50
dx = L / Nx
T = 0.2
Nt = 500
dt = T / Nt
x = np.linspace(0, L, Nx)
t = np.linspace(0, T, Nt)
x.shape, t.shape

((50,), (500,))

In [7]:
X, T = np.meshgrid(x, t)
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
xtrue = torch.tensor(xtrue, dtype=torch.float32, device="cuda")
device = "cuda"

In [ ]:
import torch
import numpy as np
def loadAndPrep(u):
    idx = np.random.choice(u.flatten().shape[0], 10000, replace=False)
    global xtrue
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    utrain = torch.tensor(utrain, dtype=torch.float32, device=device)
    return xtrain, utrain

In [9]:
import gc
def trainAndLog(u, e, utrue):
    xtrain, utrain = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1, alpha=e).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=5000)
    with torch.no_grad():
        u_pred = model(xtrue).cpu().numpy()
        mse = np.mean((u_pred - utrue.flatten()[:, None]) ** 2)
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return loss, mse

In [ ]:
import random
import json

random.seed(69)
d = {}
with h5py.File("data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=50)
    n = 0
    for i in a:
        print(n)
        e = f[i]["alpha"][()]
        print(e)
        uclean = f[i]["u"][:].T
        loss, mse = trainAndLog(uclean, e, uclean)
        unoisy = f[i]["u_noisy"][:]
        loss1, mse1 = trainAndLog(unoisy, e, uclean)
        d[i] = {
            "clean": {"loss": float(loss), "mse": float(mse)},
            "noisy": {"loss": float(loss1), "mse": float(mse1)},
        }
        n += 1
        with open("results.json", "w") as g:
            json.dump(d, g)